In [1]:
import os
import sys

# Init project path
PROJECT_DIR = os.getcwd() + "/../"
sys.path.insert(0, PROJECT_DIR)
%load_ext autoreload
%autoreload 2

In [2]:
import json

config: dict = json.load(open("./../configs/lightgbm_0.json"))

In [3]:
from db import table_load

# Load data
train = table_load(table_name=config["dataset"]["train_table"])
test = table_load(table_name=config["dataset"]["test_table"])

In [69]:
import pandas as pd
from db import find_table_name
table_names = []
table_names += find_table_name(like="train")["table_name"].to_list()
table_names += find_table_name(like="test")["table_name"].to_list()
table_names += find_table_name(like="cv_result")["table_name"].to_list()

In [137]:
from db import table_write, table_load

for table_name in table_names:
    df = table_load(table_name=table_name)
    stats = pd.concat([
        df.dtypes.rename("dtype").astype(str).to_frame(),
        df.isnull().sum().rename("null_count").to_frame(),
        df.describe().T.rename(columns={"25%": "per_25", "50%": "per_50", "75%": "per_75"}),
    ], axis=1, sort=False)
    table_write(
        table_name="{}_stats".format(table_name),
        df=stats,
    )